In [1]:
import sys
sys.path.append("../")

In [2]:
from data import load_gyafc
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import (DebertaTokenizer,
                          DebertaForSequenceClassification,
                          AutoModelForSequenceClassification,  
                          AutoTokenizer,
                          Trainer,TrainingArguments)
import os
import json
from tqdm import tqdm

2023-05-09 17:16:46.854965: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 17:16:47.633392: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from mt5_utils import MT5ForSequenceClassification

In [4]:
import transformers

In [5]:
transformers.__version__

'4.28.1'

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [7]:
def get_test_dataset(model_name):
    tr_val_test_datasets = load_gyafc(model_name, toy = False)
    _, _, test_dataset = tr_val_test_datasets
    return test_dataset

In [22]:
def get_model_type(model_name):
    if "deberta-large" in model_name:
        return "microsoft/deberta-large"
    elif "deberta-base" in model_name:
        return "microsoft/deberta-base"
    elif "bigscience_bloom-560m" in model_name:
        return "bigscience/bloom-560m"
    elif "mt5-base" in model_name:
        return "google/mt5-base"
    
    elif "distilbert-base-multilingual-cased" in model_name:
        return "distilbert-base-multilingual-cased"
    
    elif "bert-base-multilingual-cased" in model_name:
        return "bert-base-multilingual-cased"

In [16]:
test_dataset_dict = {}
# for mt in ["microsoft/deberta-large","microsoft/deberta-base","bigscience/bloom-560m","google/mt5-base"]:
#     test_dataset_dict[mt] = get_test_dataset(mt)
    
for mt in ["distilbert-base-multilingual-cased"]:
    test_dataset_dict[mt] = get_test_dataset(mt)

In [18]:
test_dataset_dict

{'distilbert-base-multilingual-cased': <data.Formal_informal at 0x7ff4d8cc4610>}

In [19]:
model_type

'bert-base-multilingual-cased'

In [23]:
trained_models_fld = "../trained_models/"

existing_inference_results = os.listdir("test_results_trainer/")

for model_folder in os.listdir(trained_models_fld) :
    
    if f"{model_folder}.json" not in existing_inference_results and model_folder != ".ipynb_checkpoints":
        
        print(model_folder)
        
        model_type = get_model_type(model_folder)
        current_test_dataset = test_dataset_dict[model_type]      
        
        model_folder_abs = os.path.join(trained_models_fld, model_folder)
        try:
            test_model = AutoModelForSequenceClassification.from_pretrained(f"{model_folder_abs}/nli_model/")
        except ValueError:
            test_model = MT5ForSequenceClassification.from_pretrained(f"{model_folder_abs}/nli_model/")
        
        training_args = TrainingArguments(per_device_eval_batch_size=64, output_dir = "./tmp_trainer/", dataloader_drop_last = True)
        
        trainer = Trainer(model=test_model, args = training_args)
        
        trainer_preds = trainer.predict(current_test_dataset)
        
        list_predited_label = torch.max(torch.tensor(trainer_preds.predictions), dim=1).indices.tolist()    
        
        with open(f"test_results_trainer/{model_folder}.json", "w") as f:
                json.dump(list_predited_label, f)
    else:
        print("dropped", model_folder)

dropped bert-base-multilingual-cased_ep5_wus1598_lr1e-06_batch32
dropped google_mt5-base_ep5_wus2046_lr1e-05_batch25
dropped microsoft_deberta-large_ep5_wus2000_lr1e-05
dropped bigscience_bloom-560m_ep5_wus6394_lr1e-06_batch8
dropped microsoft_deberta-large_ep5_wus12789_lr1e-06_batch16
dropped google_mt5-base_ep5_wus6394_lr1e-05_batch16
distilbert-base-multilingual-cased_ep5_wus1598_lr1e-06_batch32
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environme

dropped microsoft_deberta-base_ep10_wus2000_lr0.0001
dropped bigscience_bloom-560m_ep5_wus12789_lr1e-06_batch8
dropped microsoft_deberta-large_ep5_wus100_lr1e-05
dropped google_mt5-base_ep5_wus3197_lr1e-06_batch16
dropped bigscience_bloom-560m_ep3_wus6394_lr1e-05_batch8
dropped bert-base-multilingual-cased_ep5_wus1598_lr1e-05_batch32
dropped microsoft_deberta-large_ep5_wus6394_lr1e-06_batch16
dropped microsoft_deberta-large_ep5_wus19184_lr1e-06_batch16
dropped microsoft_deberta-large_ep5_wus2000_lr1e-06
dropped microsoft_deberta-large_ep5_wus100_lr1e-06
dropped bigscience_bloom-560m_ep5_wus12789_lr1e-05_batch8
dropped .ipynb_checkpoints


In [ ]:
#old vers

In [ ]:
trained_models_fld = "../trained_models/"

existing_inference_results = os.listdir("test_results/")

for model_folder in os.listdir(trained_models_fld):
    
    if f"{model_folder}.json" not in existing_inference_results:
        
        print(model_folder)
        
        model_type = get_model_type(model_folder)
        current_test_dataset = test_dataset_dict[model_type]      
        
        test_dataloader = DataLoader(
            current_test_dataset,
            batch_size=64,
            # num_workers=4,
            drop_last=True
        )
        
        model_folder_abs = os.path.join(trained_models_fld, model_folder)        
        test_model = AutoModelForSequenceClassification.from_pretrained(f"{model_folder_abs}/nli_model/")
        test_model = test_model.to(device)
        

        list_predited_label = []
        list_label = []
        with torch.no_grad():
            for d in tqdm(test_dataloader):

                input_ids = d["input_ids"].to(device) # .reshape(64, 24)
                attention_mask = d["attention_mask"].to(device)

                outputs = test_model(input_ids, attention_mask)
                logits = outputs[0]

                _, prediction = torch.max(logits, dim=1)
                targets = d["labels"].detach().numpy().tolist()
                prediction = prediction.cpu().detach().numpy().tolist()

                list_label.extend(targets)
                list_predited_label.extend(prediction)

            with open(f"test_results/{model_folder}.json", "w") as f:
                json.dump(list_predited_label, f)
    else:
        print("dropped", model_folder)

In [ ]:
# with open(f"GYAFC_test_labels.json", "w") as f:
#     json.dump(list_label, f)

# len(list_label)